# 基于经典网络架构训练图像分类模型

### 数据预处理部分：

- 数据增强：torchvision中transforms模块自带功能，比较实用
- 数据预处理：torchvision中transforms也帮我们实现好了，直接调用即可
- DataLoader模块直接读取batch数据

### 网络模块设置：

- 加载预训练模型，torchvision中有很多经典网络架构，调用起来十分方便，并且可以用人家训练好的权重参数来继续训练，也就是所谓的迁移学习
- 需要注意的是别人训练好的任务跟咱们的可不是完全一样，需要把最后的head层改一改，一般也就是最后的全连接层，改成咱们自己的任务
- 训练时可以全部重头训练，也可以只训练最后咱们任务的层，因为前几层都是做特征提取的，本质任务目标是一致的

### 网络模型保存与测试
- 模型保存的时候可以带有选择性，例如在验证集中如果当前效果好则保存
- 读取模型进行实际测试

![title](1.png)

In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torchvision
#pip install torchvision
# 使用训练好的cv模型：resnet
from torchvision import transforms, models, datasets
#https://pytorch.org/docs/stable/torchvision/index.html
import imageio
import time
import warnings
warnings.filterwarnings("ignore")
import random
import sys
import copy
import json
from PIL import Image

### 数据读取与预处理操作

In [2]:
# 指定图片数据地址
data_dir = './flower_data/'
# 训练数据集
train_dir = data_dir + '/train'
# 验证数据集
valid_dir = data_dir + '/valid'

### 制作好数据源：
- data_transforms中指定了所有图像预处理操作
- ImageFolder假设所有的文件按文件夹保存好，每个文件夹下面存贮同一类别的图片，文件夹的名字为分类的名字

In [3]:
# 数据预处理
data_transforms = {
    'train':
        # 声明在 Compose 中的命令会按照顺序进行执行
        transforms.Compose([
        # 固定输入尺寸：输入的照片数据大小肯定是不一致的，在训练前要求必须将图片都设置为一样的尺寸。
        # 在大多数模型中使用正方形的图片训练效果较好，这个地方设置为：[96, 96]是基于电脑性能考虑，这个尺寸越大训练越好，但也不要太大，
        transforms.Resize([96, 96]),
        # 数据增强（Data Augmentation）：
        #    输入数据量过小，我们可以将图片进行旋转、翻转、裁剪等操作，扩大数据集、增加多样性并且可以保证是同一张图片（数据源不变）
        transforms.RandomRotation(45),#随机旋转，-45到45度之间随机选
        # 尺寸：[96, 96] --> [64, 64]
        transforms.CenterCrop(64),#从中心开始裁剪
        # 随机水平、垂直翻转，百分之五十可能性
        transforms.RandomHorizontalFlip(p=0.5),#随机水平翻转 选择一个概率概率
        transforms.RandomVerticalFlip(p=0.5),#随机垂直翻转
        # 修改照片参数
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1),#参数1为亮度，参数2为对比度，参数3为饱和度，参数4为色相
        # 修改图片RGB参数
        transforms.RandomGrayscale(p=0.025),#概率转换成灰度率，3通道就是R=G=B

        # 以下两件事必须做
        # 输入数据格式转换为：tensor
        transforms.ToTensor(),
        # 标准化操作：使用大数据集计算出的标准差来应用到本数据集（因为本数据集较小，计算出来的标准差不准确），
        # 下边的值分别对应 RGB，三色通道的标准差
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])#均值，标准差
    ]),
    'valid': 
        transforms.Compose([
        # 验证集输入图片尺寸需要和训练集使用的输入尺寸大小保持一致
        transforms.Resize([64, 64]),
        transforms.ToTensor(),
        # 验证集使用的标准差也需要和训练集保持一致
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [4]:
# 批量处理数据集大小
batch_size = 128

# 指定图片数据以及使用的Transformer
# datasets.ImageFolder(os.path.join(data_dir, x) ： 打开 ./flower_data/train 文件读取数据
# data_transforms[x] : data_transforms[train] --> 'train' Transformer
# 验证测试集同理 打开 ./flower_data/valid 文件读取数据，使用 'valid' Transformer
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}

# 将训练集和验证集数据转换为 Tensor 格式
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'valid']}
# 计算训练集和验证集大小
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [5]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 6552
     Root location: ./flower_data/train
     StandardTransform
 Transform: Compose(
                Resize(size=[96, 96], interpolation=bilinear, max_size=None, antialias=True)
                RandomRotation(degrees=[-45.0, 45.0], interpolation=nearest, expand=False, fill=0)
                CenterCrop(size=(64, 64))
                RandomHorizontalFlip(p=0.5)
                RandomVerticalFlip(p=0.5)
                ColorJitter(brightness=(0.8, 1.2), contrast=(0.9, 1.1), saturation=(0.9, 1.1), hue=(-0.1, 0.1))
                RandomGrayscale(p=0.025)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'valid': Dataset ImageFolder
     Number of datapoints: 818
     Root location: ./flower_data/valid
     StandardTransform
 Transform: Compose(
                Resize(size=[64, 64], interpolation=bilinear, max_size=None, antialias=True)
                ToTe

In [6]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1e9fc2b0>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x1e9fc340>}

In [7]:
dataset_sizes

{'train': 6552, 'valid': 818}

### 读取标签对应的实际名字

In [8]:
# cat_to_name.json中存储每个图片文件夹（标签）的名字，分配标签名字
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [9]:
cat_to_name

{'21': 'fire lily',
 '3': 'canterbury bells',
 '45': 'bolero deep blue',
 '1': 'pink primrose',
 '34': 'mexican aster',
 '27': 'prince of wales feathers',
 '7': 'moon orchid',
 '16': 'globe-flower',
 '25': 'grape hyacinth',
 '26': 'corn poppy',
 '79': 'toad lily',
 '39': 'siam tulip',
 '24': 'red ginger',
 '67': 'spring crocus',
 '35': 'alpine sea holly',
 '32': 'garden phlox',
 '10': 'globe thistle',
 '6': 'tiger lily',
 '93': 'ball moss',
 '33': 'love in the mist',
 '9': 'monkshood',
 '102': 'blackberry lily',
 '14': 'spear thistle',
 '19': 'balloon flower',
 '100': 'blanket flower',
 '13': 'king protea',
 '49': 'oxeye daisy',
 '15': 'yellow iris',
 '61': 'cautleya spicata',
 '31': 'carnation',
 '64': 'silverbush',
 '68': 'bearded iris',
 '63': 'black-eyed susan',
 '69': 'windflower',
 '62': 'japanese anemone',
 '20': 'giant white arum lily',
 '38': 'great masterwort',
 '4': 'sweet pea',
 '86': 'tree mallow',
 '101': 'trumpet creeper',
 '42': 'daffodil',
 '22': 'pincushion flower',
 

### 加载models中提供的模型，并且直接用训练的好权重当做初始化参数
- 第一次执行需要下载，可能会比较慢，我会提供给大家一份下载好的，可以直接放到相应路径

In [10]:
# 使用训练好的模型，对训练好的模型进行微调，转换为自己的模型
# 因为图像识别的模型，在提取特征的过程中一样（边缘检测、物体识别等）
model_name = 'resnet'  #可选的比较多 ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']
#是否用人家训练好的特征来做
feature_extract = True #都用人家特征，咱先不更新

In [11]:
# 是否用GPU训练
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CUDA is not available.  Training on CPU ...


#### 模型参数要不要更新
- 有时候用人家模型，就一直用了，更不更新咱们可以自己定

In [12]:
# 设置模型参数
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        # model.parameters() 中存在每一层网络的名字、权重、偏置等数据
        for param in model.parameters():
            # param.requires_grad参数设置为False，代表不更新模型参数，全部使用之前训练的权重、偏置等
            param.requires_grad = False

In [13]:
# 选用 resnet 中 resnet18 型号的模型（网络结构）
# 18层的能快点，条件好点的也可以选152
model_ft = models.resnet18()
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 把模型输出层改成自己的

- resnet18默认是输出1000特征的模型，我们需要修改最后一层（全连接层），替换为本案例的102个特征

In [14]:
# 在models.resnet18基础上定义本案例的训练模型
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):

    # 使用models.resnet18预训练模型（不存在的话会在执行的时候自动下载）
    model_ft = models.resnet18(pretrained=use_pretrained)
    # 使用models.resnet18前17层训练好的模型，不更改其中的权重、偏置等数据
    set_parameter_requires_grad(model_ft, feature_extract)

    # 只修改最后一层（全连接层）
    # models.resnet18最后全连接层  (fc): Linear(in_features=512, out_features=1000, bias=True)
    # 修改为符合自己的特征模型: out_features: [512, 1000] --> [512, 102] (fc): Linear(in_features=512, out_features=102, bias=True
    num_ftrs = model_ft.fc.in_features # 512
    # 这一层属于自定义的，需要传递新的权重、偏置等数据，这一层 param.requires_grad = True
    model_ft.fc = nn.Linear(num_ftrs, num_classes)

    #输入大小根据自己配置来
    input_size = 64

    return model_ft, input_size

### 设置哪些层需要训练

In [15]:
model_ft, input_size = initialize_model(model_name, 102, feature_extract, use_pretrained=True)

#GPU还是CPU计算
model_ft = model_ft.to(device)

# 是否训练所有层
# 将网络结构中的每一层权重、偏置等参数全部保存
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    # 将需要训练的数据进行保存，后续传递给优化器进行参数更新
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            # 这个地方只会输出网络模型最后一层（自定义的全连接层）的权重、偏置数据
            # 因为其他层级都是使用的模型中之前的权重、偏置数据，我们都设置了 param.requires_grad = False
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [16]:
# 通过以上的方式，此时模型就是修改为我们想要的样子，除了全连接层使用的都是之前模型的，全连接层级变为了 [512, 102]
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### 优化器设置

In [17]:
# 优化器设置
# params_to_update : 上述保存的需要训练的参数（目前只修改全连接层的权重参数，其余层都被冻住，使用模型之前的训练参数）
# lr: 学习率（每次模型提升的范围【0.01】）
optimizer_ft = optim.Adam(params_to_update, lr=1e-2)
# 衰减策略：学习率每10个epoch衰减成原来的1/10 【0.01 * 0.1 = 0.01】
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
# 交叉熵，损失函数
criterion = nn.CrossEntropyLoss()

### 训练模块

In [18]:
# model: 模型
# dataloaders: 训练数据集（文件夹）
# criterion: 损失函数
# optimizer: 优化器
# num_epochs: 迭代次数
# filename: 保存文件路径
def train_model(model, dataloaders, criterion, optimizer, num_epochs, filename='best.pt'):
    # 执行每个
    since = time.time()
    # 保存训练最好的一次，训练结果可能成：正态分布结果
    best_acc = 0
    # 模型需要放入到CPU或GPU中执行
    model.to(device)
    # 训练过程中存储打印一些训练及真实的损失和指标
    train_acc_history = []
    val_acc_history = []
    train_losses = []
    valid_losses = []
    # 获取当前学习率（学习率会根据迭代进行变化）
    LRs = [optimizer.param_groups[0]['lr']]
    # 最好的那次模型，后续会变的，先初始化
    best_model_wts = copy.deepcopy(model.state_dict())
    # 一个个epoch来遍历
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # 训练和验证
        for phase in ['train', 'valid']:
            # 修改模型模式
            if phase == 'train':
                model.train()  # 训练
            else:
                model.eval()   # 验证

            # 存储每个epoch的损失率和准确率
            # 注意：每个epoch拆分为多个batch进行处理，将每个batch加起来最后算平均值
            running_loss = 0.0
            running_corrects = 0

            # 分别读训练数据集和验证数据集
            # 注意：在转换dataloaders的时候加入了批次的概念，所以这个地方可不只是循环两次
            #      分批大小是128，每次只获取128个数据进行训练或验证。
            for inputs, labels in dataloaders[phase]:
                # 将数据放入到CPU或GPU
                inputs = inputs.to(device)
                labels = labels.to(device)

                # 梯度清零
                optimizer.zero_grad()
                # 调用模型，输出结果就是102个预测值，存在批次，输出：[128, 102]
                outputs = model(inputs)
                # 调用损失函数，计算损失
                loss = criterion(outputs, labels)
                # 获取102个特征中最大的，存在批次，输出：[128] 个值
                _, preds = torch.max(outputs, 1)
                # 只有训练的时候计算和更新梯度
                if phase == 'train':
                    # 反向传播
                    loss.backward()
                    # 梯度更新（更新权重、偏置等数据）
                    optimizer.step()

                # 计算并且累加本次batch的损失率和准确率
                # 0表示batch那个维度
                running_loss += loss.item() * inputs.size(0)
                # 预测结果最大的和真实值是否一致
                running_corrects += torch.sum(preds == labels.data)
                
            
            # 计算本次epoch的平均损失率和预测准确率（每次批量数据计算出来的平均值）
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            # 计算执行一个epoch需要多少时间
            time_elapsed = time.time() - since
            print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            

            # 得到最好那次的模型
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                state = {
                  # 按照state_dict中的键（key），保存训练最好的一次的权重
                  'state_dict': model.state_dict(),
                  'best_acc': best_acc,
                  'optimizer' : optimizer.state_dict(),
                }
                # 将训练好数据保存到文件中
                torch.save(state, filename)
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append(epoch_loss)
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
        
        print('Optimizer learning rate : {:.7f}'.format(optimizer.param_groups[0]['lr']))
        # 将每个epoch的学习率进行存储
        LRs.append(optimizer.param_groups[0]['lr'])
        print()
        # 学习率衰减（注意并不是执行完一个epoch学习率都会衰减，具体执行多少个epoch衰减要看scheduler中配置的step_size参数）
        # 上述配置是step_size=10，说明执行10个epoch才会衰减一次
        scheduler.step()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # 训练完后用最好的一次当做模型最终的结果,等着一会测试
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history, valid_losses, train_losses, LRs 

### 开始训练！

- 我们现在只训练了输出层

In [ ]:
model_ft, val_acc_history, train_acc_history, valid_losses, train_losses, LRs  = train_model(model_ft, dataloaders, criterion, optimizer_ft, num_epochs=20)

Epoch 0/19
----------


### 再继续训练所有层
- 上述将全连接层训练完成，接下来使用训练好的全连接层参数
- 将原始模型的全部层进行解冻，更新整个模型的层级参数（权重、偏置）

In [ ]:
# 设置更新整个模型所有层的参数（权重、偏置）
for param in model_ft.parameters():
    param.requires_grad = True

# 重新设计优化器，需要更新全部层的参数，学习率调小一点
optimizer = optim.Adam(model_ft.parameters(), lr=1e-3)
# 衰减策略：学习率每7个epoch衰减成原来的1/10 【0.01 * 0.1 = 0.01】
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

# 损失函数
criterion = nn.CrossEntropyLoss()

In [ ]:
# 加载之前训练好的权重参数（全连接层）

checkpoint = torch.load('best.pt')
best_acc = checkpoint['best_acc']
# 模型的某一层名称与state_dict中的键（key）完全匹配，则该层的参数会被覆盖
# 当前案例中state_dict中存储的key代表的全连接层，所以全连接的权重会被替换完之前训练出来最好的那次权重参数
model_ft.load_state_dict(checkpoint['state_dict'])

In [ ]:
model_ft, val_acc_history, train_acc_history, valid_losses, train_losses, LRs  = train_model(model_ft, dataloaders, criterion, optimizer, num_epochs=10,)

### 加载训练好的模型

In [ ]:
model_ft, input_size = initialize_model(model_name, 102, feature_extract, use_pretrained=True)

# 设置使用GPU加载还是CPU加载
model_ft = model_ft.to(device)

# 保存模型文件的名字
filename='best.pt'

# 加载模型
checkpoint = torch.load(filename)
best_acc = checkpoint['best_acc']
# 加载模型中训练好的参数
model_ft.load_state_dict(checkpoint['state_dict'])

### 测试数据预处理

- 测试数据处理方法需要跟训练时一直才可以 
- crop操作的目的是保证输入的大小是一致的
- 标准化操作也是必须的，用跟训练数据相同的mean和std,但是需要注意一点训练数据是在0-1上进行标准化，所以测试数据也需要先归一化
- 最后一点，PyTorch中颜色通道是第一个维度，跟很多工具包都不一样，需要转换


In [ ]:
# 得到一个batch的测试数据
dataiter = iter(dataloaders['valid'])
images, labels = next(dataiter)

model_ft.eval()

if train_on_gpu:
    output = model_ft(images.cuda())
else:
    output = model_ft(images)

output表示对一个batch中每一个数据得到其属于各个类别的可能性

In [ ]:
output.shape

### 得到概率最大的那个

In [ ]:
_, preds_tensor = torch.max(output, 1)

preds = np.squeeze(preds_tensor.numpy()) if not train_on_gpu else np.squeeze(preds_tensor.cpu().numpy())
preds

### 展示预测结果

In [ ]:
def im_convert(tensor):
    """ 展示数据"""
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image

In [ ]:
fig=plt.figure(figsize=(20, 20))
columns =4
rows = 2

for idx in range (columns*rows):
    ax = fig.add_subplot(rows, columns, idx+1, xticks=[], yticks=[])
    plt.imshow(im_convert(images[idx]))
    ax.set_title("{} ({})".format(cat_to_name[str(preds[idx])], cat_to_name[str(labels[idx].item())]),
                 color=("green" if cat_to_name[str(preds[idx])]==cat_to_name[str(labels[idx].item())] else "red"))
plt.show()